<a href="https://colab.research.google.com/github/The-DarK-os/deep-learning/blob/master/Mnist_gans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import tensorflow as tf
print(tf.__version__)
print("GPU Avail? ",tf.test.is_gpu_available())

2.9.2
GPU Avail?  True


In [26]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print("Found GPU at {}".format(device_name))

Found GPU at /device:GPU:0


In [27]:
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [28]:
def make_generator(num_hidden_layer = 1,num_hidden_units=100,num_output_units=784):
    model = tf.keras.models.Sequential()
    for i in range(num_hidden_layer):
      model.add(tf.keras.layers.Dense(units=num_hidden_units,use_bias=False))
      model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dense(num_output_units,activation='tanh'))
    return model

def make_discriminator(num_hidden_layer=1,num_hidden_units=100,num_output_units=1):
  model = tf.keras.models.Sequential()
  for i in range(num_hidden_layer):
    model.add(tf.keras.layers.Dense(units=num_hidden_units))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(rate=0.5))
  model.add(tf.keras.layers.Dense(units=num_output_units,activation=None))
  return model

In [29]:
image_size = (28,28)
z_size=20
mode_z = 'uniform'
gen_hidden_layers =1
gen_hidden_size=100
disc_hidden_layers=1
disc_hidden_size=100
tf.random.set_seed(1)
gen_model = make_generator(num_hidden_layer=gen_hidden_layers,
                           num_hidden_units=gen_hidden_size,
                           num_output_units=np.prod(image_size))
gen_model.build(input_shape=(None,z_size))
gen_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 100)               2000      
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_6 (Dense)             (None, 784)               79184     
                                                                 
Total params: 81,184
Trainable params: 81,184
Non-trainable params: 0
_________________________________________________________________


In [30]:
disc_model = make_discriminator(num_hidden_layer=disc_hidden_layers,
                                num_hidden_units=disc_hidden_size)
disc_model.build(input_shape =(None,np.prod(image_size)))
disc_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 100)               78500     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 100)               0         
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_8 (Dense)             (None, 1)                 101       
                                                                 
Total params: 78,601
Trainable params: 78,601
Non-trainable params: 0
_________________________________________________________________


In [31]:
mnist_bldr = tfds.builder('mnist')
mnist_bldr.download_and_prepare()
mnist = mnist_bldr.as_dataset(shuffle_files=False)

In [32]:
def preprocess(ex,mode='uniform'):
  image = ex['image']
  image = tf.image.convert_image_dtype(image,tf.float32) 
  image = tf.reshape(image,[-1]) 
  image = image*2 -1.
  if mode =='uniform': 
    input_z = tf.random.uniform(shape=(z_size,),minval=-1.0,maxval=1.0)
  else:
    input_z = tf.random.normal(shape=(z_size,))
  return input_z,image

In [33]:
mnist_train = mnist['train']
mnist_train = mnist_train.map(preprocess)

In [34]:
mnist_train = mnist_train.batch(32,drop_remainder=True)
input_z,input_real = next(iter(mnist_train))

In [35]:
g_output = gen_model(input_z)

In [36]:
d_logits_real = disc_model(input_real)

In [37]:
d_logits_fake = disc_model(g_output)

In [38]:
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
g_labels_real = tf.ones_like(d_logits_fake) 
g_loss = loss_fn(y_true=g_labels_real,y_pred = d_logits_fake) 
d_labels_real = tf.ones_like(d_logits_real)
d_labels_fake = tf.zeros_like(d_logits_fake)
d_loss_real = loss_fn(y_true = d_labels_real,y_pred = d_logits_real)
d_loss_fake = loss_fn(y_true = d_labels_fake,y_pred = d_logits_fake)

In [39]:
import time
num_epochs = 100
batch_size = 64
image_size = (28,28)
z_size=10
mode_z = 'uniform'
gen_hidden_layers = 1
gen_hidden_size = 100
disc_hidden_layers = 1
disc_hidden_size = 100

In [40]:
tf.random.set_seed(1)
np.random.seed(1)

fixed_z = tf.random.uniform(shape = (batch_size,z_size),minval=-1.,maxval=1.)

In [41]:
def create_sample(g_model,z_input):
  g_output = g_model(input_z,training=False)
  images = tf.reshape(g_output,(batch_size*image_size))
  return (images+1)*2.

In [42]:
mnist_train = mnist['train']
mnist_train = mnist_train.map(lambda ex:preprocess(ex,mode=mode_z))

In [43]:
mnist_train = mnist_train.shuffle(10000)
mnist_train = mnist_train.batch(batch_size,drop_remainder=True)

In [44]:
with tf.device(device_name):
  gen_model = make_generator(gen_hidden_layers,gen_hidden_size,np.prod(image_size))
  gen_model.build(input_shape = (None,z_size))

  disc_model = make_discriminator(disc_hidden_layers,disc_hidden_size)
  disc_model.build(input_shape=(None,np.prod(image_size)))

In [48]:
opt = tf.keras.optimizers.Adam()
all_losses = []
all_d_values = []
epoch_samples = []

start_time = time.time() 
for epoch in range(1,num_epochs+1):
  epoch_losses , epoch_d_vals = [], []
  for i , (input_z,input_real) in enumerate(mnist_train):
    with tf.GradientTape() as g_tape:
      g_out = gen_model(input_z)
      g_log_fake = disc_model(g_out,training=True)
      labels_real = tf.ones_like(d_logits_fake)
      g_loss = loss_fn(y_true = labels_real,y_pred = d_logits_fake)
    g_grads = g_tape.gradient(g_loss,gen_model.trainable_variables)
    opt.apply_gradients(grads_and_vars = zip(g_grads,gen_model.trainable_variables))

ValueError: ignored